In [1]:
pip install docx

Note: you may need to restart the kernel to use updated packages.


In [ ]:
nik_defoltny = input()
nik = ('_').join(nik_defoltny.split())

import sys, subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'gspread'])
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

CREDENTIALS_FILE = 'filename.json'
myscope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
spreadsheet1_id = 'some_spreadsheet1_id'
spreadsheet2_id = 'some_spreadsheet2_id'
# Authorising and getting the service instance of API access
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, myscope)
myclient = gspread.authorize(credentials)
mysheet1 = myclient.open('diagnostics1').sheet1
mysheet2 = myclient.open('diagnostics2').sheet1

# not very elegant walkaround the error which raises in some versions of gspread when
# we use 'get_all_values()':
col_score1 = mysheet1.col_values(2)
n1 = len(col_score1)
col_score2 = mysheet2.col_values(3)
n2 = len(col_score2)
fin1 = []
for row_1 in range(1, n1 + 1):
    values_list1 = mysheet1.row_values(row_1)
    fin1.append(values_list1)
d1 = pd.DataFrame(fin1)
d1 = d1.rename(columns=d1.iloc[0]).drop(d1.index[0])
fin2 = []
for row_2 in range(1, n2 + 1):
    values_list2 = mysheet2.row_values(row_2)
    fin2.append(values_list2)
d2 = pd.DataFrame(fin2)
d2 = d2.rename(columns=d2.iloc[0]).drop(d2.index[0])

d1 = d1.drop(['Отметка времени', 'Баллы'], axis = 1)
d2 = d2.drop(['Адрес электронной почты', 'Отметка времени', 'Баллы'], axis = 1)
d1t = d1.set_index('Имя и фамилия').T
d2t = d2.set_index('Имя и фамилия').T
d1t.columns = d1t.columns.str.replace(' ', '_')
d2t.columns = d2t.columns.str.replace(' ', '_')


answers = (["answer1",
                        "answer2",
                        "answer3"
                        #here are all the correct answers to the form
])

storage = [
    [1, "column2_row1", "column3_row1"],
    [2, "column2_row2", "column3_row2"],
    [3, "column2_row3", "column3_row3"]
    #in this list of lists there is the data which will be displayed in the columns and rows
    #of the table respectively if the one makes a mistake
]

answers = pd.DataFrame(answers)
#pers_funct(x) is a function which concatinates the results from both files
#pers_funct(x) also helps to deal with the situations when one person solved only one of the two tests
def pers_funct(x):
    global person
    if x in d1t and x in d2t:
        person = pd.concat([d1t[x], d2t[x]])
        return person
    if x in d1t and x not in d2t:
        person = pd.concat([d1t[x], answers[31:]])
        return person
    if x in d2t and x not in d1t:
        person = pd.concat([answers[:31], d2t[x]])
        return person

#processing the errors caused by the situation when some respondents typed their niks like 'nik_':
try:
    person_df = pd.DataFrame(pers_funct(nik))
except KeyError:
    person_df = pd.DataFrame(pers_funct(nik + '_'))

person_df.reset_index(drop= True , inplace= True)

nt = pd.concat([person_df, answers], axis = 1)
#searching and saving the numbers of the exercises where there were made mistakes
nt.columns = ['Client', 'Correct answers']
nt['Compare'] = nt['Client'] == nt['Correct answers']
fa_defoltny = nt.index[nt['Compare'] == False].tolist()
fa = []
for num in fa_defoltny:
    fa.append(num + 1)

#WORKING WITH WORD
#setting some parameters of the table
import docx
import os
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
from docx.shared import Cm

#creating the document
doc = docx.Document()
style = doc.styles['Normal']
font = style.font
font.name = 'Arial'
header_0 = doc.add_paragraph().add_run('the_main_title')
header_0.font.size = Pt(14)
header_0.bold = True

header_1 = doc.add_paragraph()
#the second title
header_1.add_run(f'Имя: {nik_defoltny}').underline = True
#creating the first row with the columns' titles
table = doc.add_table(1, 3)                        
table.style = 'Table Grid'

table.columns[0].width = Cm(1.5)
table.columns[1].width = Cm(11)
table.columns[2].width = Cm(4)

header_cells = table.rows[0].cells
header_cells[0].text = 'excercise'
header_cells[0].width = 0.1
header_cells[1].text = 'title_of_the_second_column'
header_cells[2].text = 'title_of_the_third_column'

#biding false answers with recordings in storage
new_storage = []
for i in fa:
    for j in storage:
        if i == j[0]:
            new_storage.append(j)
            
#marking some numbers, which show the most important (!) and umimportant mistakes (*)
formatting_imp = [1, 3, 6, 7, 8, 9, 10, 11, 12, 13, 16, 19, 20, 22, 23, 27, 28, 29, 30, 31,
                 33, 34, 35, 40, 41, 42, 43, 45, 51, 52]
formatting_addit = [36, 37, 47, 48, 56, 57, 58, 59,60]
for g in new_storage:
    if g[0] in formatting_imp:
        g[0] = str(g[0])
        g[0] = '№' + g[0] + ' (!)'
    elif g[0] in formatting_addit:
        g[0] = str(g[0])
        g[0] = '№' + g[0] + '*'
    else:
        g[0] = str(g[0])
        g[0] = '№' + g[0]

#adding the data to the table from the storage for the specific person
for row in new_storage:
    cells = table.add_row().cells
    for k, item in enumerate(row):
        cells[k].text = str(item)

#setting the columns' width
for cell0 in table.column_cells(0):
    cell0.width = Cm(1.5)

for cell1 in table.column_cells(1):
    cell1.width = Cm(11)
    
for cell2 in table.column_cells(2):
    cell2.width = Cm(4)

doc.save(f'{nik}_result.docx')
#opening the file with the table
os.system(f'start {nik}_result.docx')